<a href="https://colab.research.google.com/github/HYEONJONG/98_app_building/blob/master/chatper2_DQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Chapter 2
import gymnasium as gym
env = gym.make('CartPole-v1')

env.action_space
env.action_space.n
[env.action_space.sample() for _ in range(10)]
env.observation_space   # cart position, cart velocity, pole angle, pole angular velocity
env.observation_space.shape

env.reset(seed = 100)
env.step(0)
env.step(1)             # new state, reward, terminated, truncated, additional data

# A randeom agent
class RandomAgent:
  def __init__(self):
    self.env = gym.make('CartPole-v1')
  def play(self, episode = 1):
    self.trewards=list()
    for e in range(episode):
      self.env.reset()
      for step in range(1,100):
        a = self.env.action_space.sample()
        state, reward, done, truc, info=self.env.step(a)
        if done:
          self.trewards.append(step)
          break

ra = RandomAgent()
ra.play(15)
ra.trewards
round(sum(ra.trewards)/len(ra.trewards),2)

24.07

In [20]:
#